### Distributed Neural Networks

How to work with Cross Validation, Neural Networks and Clusters:

In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import warnings
from dask.distributed import Client
import os
import numpy as np
import time




We create a simple binary classification dataset:

In [ ]:
X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
print("Example of datset row: "+str(X[:1]))

Let's define the ```Neural Network```:

In [ ]:
def build_neural_network():
    
    # create model
    model = MLPClassifier(hidden_layer_sizes=(100, 200, 100), activation='relu', solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001)
    return model


Prepare the cross validation data splits.

In [ ]:
data_split = []
kfold = StratifiedKFold(5, shuffle=True, random_state=42)
for train, test in kfold.split(X, y):
    data_split.append((train, test))

Create our ```classifier``` instance:

In [ ]:
clf = build_neural_network()

Train the neural network classifier for each dataset train split and test the trained model on each test dataset split, and in the end we calculate the mean of the results.

In [ ]:
start_time = time.time()
results = []
for train_idx, test_idx in data_split:
    clf.fit(X[train_idx], y[train_idx])
    Y_pred = clf.predict(X[test_idx])
    results.append(accuracy_score(y[test_idx], Y_pred))
end_time = time.time()    
print("Mean of the results: "+str(np.array(results).mean())+" in: "+str(end_time-start_time)+"s")

The process have took several seconds, but wath change if we use a cluster instead? Let's see:

First, let create the cluster and share the dataset to it:

In [ ]:

client = Client()
client.scatter(X)
client.scatter(y)

Then, let's define the process that will be distributed:

In [ ]:
def distribute_cross_validation(args):
    train_idx, test_idx = args
    clf = build_neural_network()
    clf.fit(X[train_idx], y[train_idx])

    y_pred = clf.predict(X[test_idx])
    print(str(accuracy_score(y[test_idx], y_pred)))
    return accuracy_score(y[test_idx], y_pred)

Start the process and retrieve the results:

In [ ]:
start_time = time.time()

futures = client.map(distribute_cross_validation, [(train_idx, test_idx) for train_idx, test_idx in data_split])
results = client.gather(futures)

end_time = time.time()   
print("Mean of the results: "+str(np.array(results).mean())+" in: "+str(end_time-start_time)+"s")

The process took ~1/5 of the total time. In general approaches when you have to validate a reliable machine learning model, this process should be reapeated a huge number of times with random splits of data. Using cluster machines is essential to work in a proper way. Nowadays, the improvements of the deep models and the increase of the available data (big data) has further increase those necessities.

Note: This is a simple approach, but clusters are extremely useful even when the ml algorithms can be parllelized. Apache Spark, that we will see in last lecture,  exploit those ideas.